In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

import torch
import transformers
import baukit
from tqdm.auto import tqdm
import json
import os
from src import functional
import numpy as np
import logging
from src import models

from src.utils import logging_utils, env_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__, transformers.__version__, torch.version.cuda

/home/local_arnab/miniconda3/envs/sae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('2.3.1', '4.43.3', '12.1')

In [3]:
from src.models import ModelandTokenizer

# model_name = "openai-community/gpt2-xl"
# model_name = "openai-community/gpt2"
model_name = "EleutherAI/pythia-410m"

mt = ModelandTokenizer(
    model_key=model_name,
    # torch_dtype=torch.float16,
)

mt.n_embd, mt.n_layer

2024-08-22 14:46:42 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
2024-08-22 14:46:42 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/EleutherAI/pythia-410m> | size: 875.114 MB | dtype: torch.float16 | device: cuda:0


(1024, 24)

In [7]:
from dictionary_learning import ActivationBuffer
from dictionary_learning.training import trainSAE
from src.functional import get_module_nnsight
from dictionary_learning.dictionary import AutoEncoder

submodule = get_module_nnsight(mt, mt.layer_name_format.format(mt.n_layer//2))
activation_dim = mt.n_embd
dictionary_dim = 4096

dataset_name = "rand_2"
save_dir = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_saes",
    model_name.split("/")[-1],
    dataset_name,
    "trainer_0"
)
os.makedirs(save_dir, exist_ok=True)

# rand_ae = AutoEncoder(
#     activation_dim=activation_dim,
#     dict_size = dictionary_dim
# )
# torch.save(rand_ae.state_dict(), os.path.join(save_dir, "ae.pt"))

In [8]:
from dictionary_learning.dictionary import AutoEncoder
from src.utils import env_utils

ae = AutoEncoder.from_pretrained(
    path = os.path.join(save_dir, "ae.pt"),
    device='cuda:0'
)

In [9]:
ae.encoder.weight

Parameter containing:
tensor([[ 0.0104,  0.0311, -0.0214,  ...,  0.0170, -0.0002,  0.0222],
        [ 0.0180, -0.0082,  0.0032,  ...,  0.0098,  0.0042,  0.0009],
        [ 0.0227,  0.0072,  0.0311,  ...,  0.0126,  0.0057,  0.0223],
        ...,
        [-0.0273,  0.0209,  0.0192,  ...,  0.0186,  0.0068, -0.0100],
        [-0.0156,  0.0038, -0.0150,  ..., -0.0089,  0.0303, -0.0053],
        [ 0.0116,  0.0042,  0.0184,  ..., -0.0006, -0.0157, -0.0311]],
       device='cuda:0', requires_grad=True)